In [112]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def predict_clossing_price(start_year, start_month, start_day, code):
    
    start = datetime.datetime(start_year,start_month,start_day)
    end = datetime.date.today()
    stockData = data.DataReader(code , 'yahoo',start,end)

    def get_rsi(sdata,m,mem):
        neg = 0
        pos = 0
        RS = 0
        RSI = 100

        upcloses = 0
        downcloses = 0

        n = m
        k = m-1

        for p in range(mem):
           diff = sdata[n,3] - sdata[k,3]
           if (diff>=0):
            upcloses = upcloses + diff
            pos = pos+1
           else:
            downcloses = downcloses + diff
            neg = neg+1

           n = n-1
           k = k-1

        downcloses = -downcloses
        if(neg == 0):
            return 100
        else:
            RS = (upcloses*neg)/(downcloses*pos)


        RSI = 100 - (100/(1+RS))

        return RSI


    def get_mfi(sdata,m,mem):
        neg = 0
        pos = 0
        MFR = 0
        MFI = 100

        pmflow = 0
        nmflow = 0

        n = m
        k = m-1

        for p in range(mem):

           typ_pricec = (sdata[n,0] + sdata[n,1] + sdata[n,3])/3
           typ_pricep = (sdata[k,0] + sdata[k,1] + sdata[k,3])/3

          # print(typ_price,sdata[n,0],sdata[n,1],sdata[n,3])

           if (typ_pricec>=typ_pricep):
             pmflow = pmflow + ((sdata[n,4])*(typ_pricec))
             pos = pos+1
           else:
             nmflow = nmflow + ((sdata[n,4])*(typ_pricec))
             neg = neg+1

           n = n-1
           k = k-1

        if(neg == 0):
            return 100
        else:
            MFR = pmflow/nmflow

        MFI = 100 - (100/(1+MFR))

        return MFI

    def get_ema(sdata,m,mem,EMAp):

       EMA = sdata[m,3]*(2/(1+mem)) + (1-(2/(1+mem)))*EMAp 
       #print("EMA",EMA)

       return EMA

    def get_so(sdata,m,mem):

       SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100
       #print("SO",SO)
       return SO


    memory = 14
    sdat2 = stockData.reset_index()
    del sdat2["Date"]
    del sdat2["Adj Close"]
    sdat3 = np.array(sdat2,dtype=np.float32)

    #print(sdat3[0])

    df1 = pd.DataFrame(columns=['Open','High','Low','Close','Volume','RSI','MFI','EMA','SO','CloseNext'])

    df2 = pd.DataFrame(columns=['Close','RSI','MFI','EMA','SO','CloseNext'])

    arr = np.array(df1.values)

    #print("Printing j\n")

    EMAp = 0
    acc = 0

    for i in range(memory):
        acc = acc + sdat3[i,3]

    EMAp = acc / memory    

    #for i in range(len(sdat3) -memory):
    for i in range(len(sdat3)-1 -memory):
        j = i + memory

        RSI = get_rsi(sdat3,j,memory)
        #print("RSI:",RSI)

        MFI = get_mfi(sdat3,j,memory)
        #print("MFI:",MFI)

        EMA = get_ema(sdat3,j,memory,EMAp)
        EMAp = EMA

        SO = get_so(sdat3,j,memory)

        N_close = sdat3[j+1,3]

        rec1 = [sdat3[j,2],sdat3[j,0],sdat3[j,1],sdat3[j,3],sdat3[j,4],RSI,MFI,EMA,SO,N_close]
        rec2 = [sdat3[j,3],RSI,MFI,EMA,SO,N_close]

        if(sdat3[j,4]!=0):
            d1 = {"Open":sdat3[j,2],"High":sdat3[j,0],"Low":sdat3[j,1],"Close":sdat3[j,3],"Volume":sdat3[j,4],"RSI":RSI,"MFI":MFI,"EMA":EMA,"SO":SO,"CloseNext":N_close}
            df1.loc[i] = rec1
            df2.loc[i] = rec2

    #ANN
    dataset = df1
    y = pd.DataFrame(dataset['CloseNext'])
    X = dataset.drop(['CloseNext'], axis = 1)

    X = np.array(X)
    y = np.array(y)
    #X = X[1700:2030,:]
    #y = y[1700:2030,:]
    y = y.flatten()

    #Feature scaling
    scaled = StandardScaler()
    scaled.fit(X)
    X = scaled.transform(X)

    #Train Test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    ## ANN implementation
    # define base model of our neural network for regression taks
    def endgame():
        # Adding the neurons in various layers
        model = Sequential()
        model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='relu'))
        model.add(Dense(15, kernel_initializer='normal', activation='relu'))
        model.add(Dense(8, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model for our use in KerasRegressor
        model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mape'])
        return model

    ann_regression = KerasRegressor(build_fn = endgame, epochs=100, batch_size=5, verbose=1)

    ann_regression.fit(X_train,y_train)

    ann_predict = ann_regression.predict(X_test)

    error = mean_absolute_error(ann_predict,y_test)
    per_err = (error/np.mean(y_test)) * 100
    print('The mean absolute error is {} and percentage error is {}.'.format(error,per_err))
    
    return ann_regression.predict(scaled.transform([df1.iloc[-1,:-1].values.tolist()])), df1.iloc[-1,:], error,per_err

In [113]:
#parameter
start_year = 2011
start_month = 1
start_day = 1
code = "IRFC.NS"

#predicted_price, df1  = predict_clossing_price(start_year, start_month, start_day, code)

In [114]:
list_of_stock_code = ["IOC.NS", "INFY.NS", "HCLTECH.NS", "SBIN.NS", "SBICARD.NS", "SBILIFE.NS", "ITC.NS", "TECHM.NS", 
                     "RELIANCEPP-E1.NS", "WIPRO.NS", "RELIANCE.NS", "IBM", "GOOG", "HINDUNILVR.NS", "HINDPETRO.NS",
                      "HINDALCO.NS", "HDFCAMC.NS", "HDFC.NS", "BANKBARODA.NS", "YESBANK.NS", "ASHOKLEY.NS", "MOTHERSUMI.NS",
                      "IRCTC.NS", "BEPL.NS", "TATAPOWER.NS", "BAJFINANCE.NS", "ONGC.NS", "SUNPHARMA.NS", "COALINDIA.NS", 
                      "ADANIGREEN.NS", "DRREDDY.NS", "JUBLFOOD.NS", "WESTLIFE.NS", "TCS.NS", "TATACONSUM.NS", "SRTRANSFIN.NS"]

list_of_stock_code = np.unique(list_of_stock_code)

#parameter
start_year = 2011
start_month = 1
start_day = 1

data_values = []

error_code = []
for code in list_of_stock_code:
    try:
        predicted_price, df1, error,per_err  = predict_clossing_price(start_year, start_month, start_day, code)
        data_values.append([
            code, 
            round(float(df1.Close),2),
            round(float(predicted_price),2),
            round(float(error),2),
            round(float(per_err),2),
            0])
    except:
        error_code.append(code)

<ipython-input-112-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
93/93 [==============================] - 1s 2ms/step - loss: nan - mape: nan   
Epoch 2/100
93/93 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 3/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 4/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 5/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 6/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 7/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 8/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 9/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 10/100
93/93 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 11/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 12/100
93/

93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 93/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 94/100
93/93 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 95/100
93/93 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 96/100
93/93 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 97/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 98/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 99/100
93/93 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 100/100
40/40 [==============================] - 0s 1ms/step


<ipython-input-112-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 2ms/step - loss: 5572.5228 - mape: 92.7314
Epoch 2/100
349/349 [==============================] - 0s 1ms/step - loss: 344.7165 - mape: 26.7638
Epoch 3/100
349/349 [==============================] - 0s 1ms/step - loss: 132.2672 - mape: 17.7950
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: 51.6266 - mape: 13.9309
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 23.0760 - mape: 9.9949
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: 11.2120 - mape: 6.7146
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: 6.8334 - mape: 4.5689
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: 5.5906 - mape: 3.4530
Epoch 9/100
349/349 [==============================] - 1s 1ms/step - loss: 4.5870 - mape: 3.0360
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: 4.2882 - mape: 2.6537
Epoch 11/100
34

...

Epoch 1/100
349/349 [==============================] - 1s 1ms/step - loss: 43849.6837 - mape: 95.3096
Epoch 2/100
349/349 [==============================] - 0s 1ms/step - loss: 6295.5780 - mape: 34.4692
Epoch 3/100
349/349 [==============================] - 0s 1ms/step - loss: 1018.0497 - mape: 12.7133
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: 294.6262 - mape: 6.9826
Epoch 5/100
349/349 [==============================] - 0s 1ms/step - loss: 127.4507 - mape: 4.3623
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: 68.9434 - mape: 3.1595
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: 35.6419 - mape: 2.2145
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: 27.0538 - mape: 1.9406
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: 22.5556 - mape: 1.7459
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: 18.3178 - mape: 1.5349
Epoch 1

349/349 [==============================] - 0s 1ms/step - loss: 16.1875 - mape: 1.2788
Epoch 84/100
349/349 [==============================] - 0s 1ms/step - loss: 16.6259 - mape: 1.2965
Epoch 85/100
349/349 [==============================] - 0s 1ms/step - loss: 18.3677 - mape: 1.3952
Epoch 86/100
349/349 [==============================] - 0s 1ms/step - loss: 16.0951 - mape: 1.3422
Epoch 87/100
349/349 [==============================] - 0s 1ms/step - loss: 17.3145 - mape: 1.3456
Epoch 88/100
349/349 [==============================] - 0s 1ms/step - loss: 17.5296 - mape: 1.3261
Epoch 89/100
349/349 [==============================] - 0s 1ms/step - loss: 14.8066 - mape: 1.2603
Epoch 90/100
349/349 [==============================] - 0s 1ms/step - loss: 16.1282 - mape: 1.3317
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: 20.6339 - mape: 1.4425
Epoch 92/100
349/349 [==============================] - 0s 1ms/step - loss: 14.1333 - mape: 1.2894
Epoch 93/100
349/349 [=

<ipython-input-112-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 1ms/step - loss: nan - mape: nan
Epoch 2/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 3/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 5/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 11/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: n

349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 92/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 93/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 94/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 95/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 96/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 97/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 98/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 99/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 100/100
150/150 [==============================] - 0s 995us/step


In [115]:
dataset = pd.DataFrame(columns = ["symbol", "Clossing_Price", "Predicted_Price","Error", "Error_in_per", "Actual_Price"], data = data_values)
dataset.to_csv("stock_price_prediction_17_march_2021.csv")

# Storing the actual values

In [104]:
import pandas as pd
report = pd.read_csv("stock_price_prediction_17_march_2021.csv")

In [105]:
report = report.drop(["Unnamed: 0"], axis=1)

In [106]:
data_values = []
error_code = []

for code in report.symbol:
    try:
        #start = datetime.datetime(2021,3,13)
        #end = datetime.datetime(2021,3,15)
        today = datetime.date.today()
        stockData = data.DataReader(code , 'yahoo',start, end)
        data_values.append([int(stockData.High), int(stockData.Low)])
    except:
        data_values.append([0,0])

In [107]:
stockData

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-03-12,127.68,126.610001,127.190002,127.610001,4009600,127.610001
2021-03-15,128.75,127.540001,127.769997,128.580002,3420600,128.580002


In [108]:
len(data_values)

30

In [110]:
#del report["Actual_Price"]

report["Actual_High"] = [high[0] for high in data_values]
report["Actual_Low"] = [low[1] for low in data_values]

data_values_tf = (report.Predicted_Price <= report.Actual_High , report.Actual_Low <= report.Predicted_Price)

tf = []
for i,j in zip(data_values_tf[0], data_values_tf[1]):
    if all([i,j]):
        tf.append(True)
    else:
        tf.append(False)
        
report["Result"] = tf

In [111]:
report.to_csv("stock_price_prediction_17_march_2021.csv")